In [1]:
import pandas as pd
df = pd.read_csv('Anne_DNA_66.csv')
df = df[df['Class'] != 'interphase']

In [2]:
y = df['Class'].values

In [3]:
df = df.drop('TableNumber', 1)
df = df.drop('ImageNumber', 1)
df = df.drop('ObjectNumber', 1)
df = df.drop('Class', 1)
df = df.drop('Nuclei_AreaShape_EulerNumber', 1)

In [4]:
df_norm = (df - df.mean()) / (df.max() - df.min())
df_norm.values.shape

(1306, 97)

In [ ]:
y

In [5]:
# Convert y into Y
nb_classes = 23
import numpy as np
from keras.utils import np_utils, generic_utils

# Convert labels to numeric
y_unique = np.unique(y)
dic = {}

for i, label in enumerate(y_unique):
    dic[label] = i
print dic

y_numeric = []
for el in y:
    y_numeric += [dic[el]]
    
y_numeric # now a 2000 label vector
Y = np_utils.to_categorical(y_numeric, nb_classes)

print Y.shape

Y_train = Y

{'halfcircle': 7, 'metaphase': 14, 'earlyprophase': 4, 'debris': 3, 'nucleolirim': 18, 'kidney': 10, 'telophase': 21, 'blurry': 2, 'fragmented': 6, 'apoptotic': 1, 'multinucleate': 17, 'latepro_earlyana': 11, 'prophase': 19, 'anaphase': 0, 'indented': 9, 'monopole': 16, 'holey': 8, 'latetelophase': 12, 'lines': 13, 'micronucleus': 15, 'round': 20, 'elongated': 5}
(1306, 23)


In [6]:
X_train = df_norm.values

In [8]:
from sklearn.cross_validation import StratifiedKFold

#print y_numeric
skl = StratifiedKFold(y_numeric, n_folds=5)

y_numeric = np.array(y_numeric)

for train,test in skl:
    print len(train), len(test)
    
X_4 = X_train[train]
Y_4 = Y[train]
y_numeric_4 = y_numeric[train]

X_eval = X_train[test]
Y_eval = Y[test]
y_numeric_eval = y_numeric[test]

X_stratified = np.append(X_4,X_eval,axis=0)
Y_stratified = np.append(Y_4,Y_eval,axis=0)

X_stratified.shape
Y_stratified.shape

1036 270
1039 267
1046 260
1049 257
1054 252


(1306, 23)

In [ ]:
## Convolutional Neural Network with 2 convolutions

from keras.models import Sequential
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad, Adam
from keras.layers.normalization import BatchNormalization

nb_epoch = 12

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=97))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='sgd')

In [ ]:
history = model.fit(X_stratified, Y_stratified, nb_epoch=5000,validation_split=0.2, show_accuracy=True, verbose=2)

In [9]:
import skflow
from sklearn import datasets, metrics

clf = skflow.TensorFlowDNNClassifier(hidden_units=[64,256,4096], n_classes=22, steps=5000, batch_size=len(X_4))
clf.fit(X_4, y_numeric_4, logdir='/tmp/tf_examples/model_without_dropout/')

ResourceExhaustedError: OOM when allocating tensor with shape[1054,4096]
	 [[Node: dnn/layer2/add = Add[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"](dnn/layer2/Linear/MatMul, dnn/layer2/Linear/Bias/read)]]
	 [[Node: gradients/logistic_regression/softmax_classifier/xent_grad/range_1/_19 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_160_gradients/logistic_regression/softmax_classifier/xent_grad/range_1", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'dnn/layer2/add', defined at:
  File "/home/ubuntu/anaconda2/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/ubuntu/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", l

In [ ]:
score = metrics.accuracy_score(y_numeric_eval, clf.predict(X_eval))
print("Accuracy: %f" % score)

In [ ]:
from sklearn import datasets, metrics
iris = datasets.load_iris()
iris.target.shape

In [ ]:
# Creates a graph.
import tensorflow as tf
with tf.device('/cpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [10]:
clf = skflow.TensorFlowDNNClassifier(hidden_units=[64,256,8096], n_classes=22, steps=5000, batch_size=len(X_4), 
                                     learning_rate=0.05, early_stopping_rounds=True)
clf.fit(X_4, y_numeric_4, logdir='/tmp/tf_examples/model_without_dropout/')

Performing early stopping. 


ResourceExhaustedError: OOM when allocating tensor with shape[1054,8096]
	 [[Node: dnn/layer2/Linear/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](dnn/layer1/Relu, dnn/layer2/Linear/Matrix/read)]]
	 [[Node: logistic_regression/softmax_classifier/range/_23 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_180_logistic_regression/softmax_classifier/range", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'dnn/layer2/Linear/MatMul', defined at:
  File "/home/ubuntu/anaconda2/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/ubuntu/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__mai

In [ ]:
df.columns.values